<a href="https://colab.research.google.com/github/lucasgneccoh/BDSS_Dauphine/blob/main/notebooks/solutions/BDSS_TD7_PostgreSQLJSON_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bases de données semi-structurées - TD 7 - PostgreSQL and JSON

Main teacher: **Dario COLAZZO**

Teaching Assistant: **Lucas GNECCO**

Special thanks to **Beatrice NAPOLITANO**

Université Paris Dauphine - PSL

# Introduction

Welcome!

In this notebook we will practice SQL while dealing with data in JSON format. To do so we will rely on PostgreSQL which has the capacity of dealing with such data.

PostgreSQL has many in-built functions to handle JSON objects and its syntax allows to work with JSON data in a very familiar SQL-like way

For more documentation please visit the official site

https://www.postgresql.org/docs/9.3/functions-json.html


***NOTE: This notebook was designed to be executed in Google Colab. Instructions below install PostgreSQL and use other tricks that were only tested in this environment***


# Working with JSON in PostgreSQL

In this notebook we will be using very simple datasets where each row contains a JSON object. We are by no means limited to that and we could have tables with "normal" columns and JSON columns all together. 

When dealing with JSON objects in PostgreSQl we have to use a special notation

Imagine that the columns *info* contains a JSON object in each row. If one row is {'foo': 1, 'bar': [5, 6]}, then we can access the diferent fields using the operators -> and ->>

For example *info* -> 'foo' would give 1, and *info* -> 'bar' would give the JSON array [5, 6].


For more details and examples I suggest the folowing resources

https://www.postgresqltutorial.com/postgresql-json/

https://www.postgresql.org/docs/current/functions-json.html

# Database setup

## Install PostgreSQL


In [3]:
import json
import re

# install
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

## Create tables and insert data
Get the JSON version of the dataset
Then insert it in the PostgreSQL tables

In [20]:
!wget "https://raw.githubusercontent.com/lucasgneccoh/BDSS_Dauphine/main/data/films.json"
FILMS = json.load(open("films.json", "r"))
print(*FILMS.keys())

--2022-02-24 09:21:07--  https://raw.githubusercontent.com/lucasgneccoh/BDSS_Dauphine/main/data/films.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39243 (38K) [text/plain]
Saving to: ‘films.json.1’

films.json.1        100%[===================>]  38.32K  --.-KB/s    in 0s      

2022-02-24 09:21:07 (111 MB/s) - ‘films.json.1’ saved [39243/39243]

arrArtistes arrFilms


In [13]:
%%sql

DROP TABLE IF EXISTS artistsSQL;
DROP TABLE IF EXISTS filmsSQL; 

CREATE TABLE filmsSQL (
	id serial NOT NULL PRIMARY KEY,
	data json NOT NULL
);
CREATE TABLE artistsSQL (
	id serial NOT NULL PRIMARY KEY,
	data json NOT NULL
);

In [4]:
# Be careful with the ' character
a = "retrouve l'un de ses"
b = re.sub("\'","''", a)
print(b)

retrouve l''un de ses


In [5]:
%%capture

for f in FILMS["arrArtistes"]:
    json_string = re.sub("\'","''", json.dumps(f))
    cmd = f'''INSERT INTO artistsSQL (data) VALUES('{json_string}')'''
    %sql $cmd;

for f in FILMS["arrFilms"]:
    json_string = re.sub("\'","''", json.dumps(f))
    cmd = f'''INSERT INTO filmsSQL (data) VALUES('{json_string}')'''
    %sql $cmd;

## Make simple SELECT statements to see if the data is right

In [6]:
%%sql
SELECT *
FROM artistsSQL
LIMIT 3
;

 * postgresql+psycopg2://@/postgres


,id,data
0,1,"{'ACTNOM': 'Cameron', 'ACTPNOM': 'James', 'ANN..."
1,2,"{'ACTNOM': 'Hitchcock', 'ACTPNOM': 'Alfred', '..."
2,3,"{'ACTNOM': 'Scott', 'ACTPNOM': 'Ridley', 'ANNE..."


In [7]:
%%sql
SELECT data -> 'ACTNOM' as nom,
        data -> 'ACTPNOM' as prenom,
        data -> 'ANNEENAISS' as anneN
FROM artistsSQL
WHERE CAST(data ->> 'ANNEENAISS' as INTEGER) > 1950
LIMIT 5
;

 * postgresql+psycopg2://@/postgres


,nom,prenom,annen
0,Cameron,James,1954
1,Travolta,John,1954
2,Cage,Nicolas,1964
3,Burton,Tim,1958
4,Willis,Bruce,1955


# Exercises

Now that we have our data in PostgreSQL we can do a lot of things!

We can do almost everything we know in standard SQL if we are able to create the right tables from our JSON data.

On top of that, PostgreSQL has a lot of functions to deal with JSON objects that make this approach a lot easier and powerful.

Here are some examples and documentation that can be helpful:


https://www.postgresqltutorial.com/postgresql-json/

https://www.postgresql.org/docs/current/functions-json.html



## Exercise 1: Simple queries we already know

Do queries 2, 4 and 8. They should not be that hard

Query 2: Films released in 1980

Query 4: Films with Bruce Willis in it

Query 8: Role of Harvey Keitel in Reservoir Dogs


If you want something a bit more challenging, try to do someething general. For example in Query 4, what if I want to look for some other artist?

In [ ]:
# Query 2: Films released in 1980

%%sql
SELECT data -> 'TITRE' -> 'title' as title,
        CAST(data ->> 'Annee' as INTEGER) as year
FROM filmsSQL
WHERE CAST(data ->> 'Annee' as INTEGER) = 1980
;

In [ ]:
# Query 4: Films with Bruce Willis in it
%%sql
SELECT  title, year,
        roles -> 'INTITULE' as intitule
FROM (
    SELECT  data -> 'TITRE' -> 'title' as title,
            CAST(data ->> 'Annee' as INTEGER) as year,
            json_array_elements ( data -> 'ROLES' )  as roles
    FROM filmsSQL
    
) AS TMP

WHERE LOWER(roles ->> 'NOM') LIKE 'willis' AND LOWER(roles ->> 'PRENOM') LIKE 'bruce' 
;

In [ ]:
prenom, nom = 'bruce', 'willis'

query = f"""
SELECT  title, year,
        roles -> 'INTITULE' as intitule
FROM (
    SELECT  data -> 'TITRE' -> 'title' as title,
            CAST(data ->> 'Annee' as INTEGER) as year,
            json_array_elements ( data -> 'ROLES' )  as roles
    FROM filmsSQL
    
) AS TMP

WHERE LOWER(roles ->> 'NOM') LIKE '{nom}' AND LOWER(roles ->> 'PRENOM') LIKE '{prenom}' 
"""

%sql $query;

In [ ]:
# Query 8: Role of Harvey Keitel in Reservoir Dogs

%%sql
SELECT  title, year,
        roles -> 'INTITULE' as intitule
FROM (
    SELECT  data -> 'TITRE' -> 'title' as title,
            CAST(data ->> 'Annee' as INTEGER) as year,
            json_array_elements ( data -> 'ROLES' )  as roles
    FROM filmsSQL
    WHERE LOWER(data -> 'TITRE' ->> 'title') LIKE 'reservoir dogs'
    
) AS TMP

WHERE LOWER(roles ->> 'NOM') LIKE 'keitel' AND LOWER(roles ->> 'PRENOM') LIKE 'harvey' 
;

## Exercise 2: More complex queries we have maybe talked about

Let's use the nice SQL syntax to JOIN the two tables we have (films and artists)

### Exercise 2.1: For every movie, show its title, year and the information about the director

In [ ]:
# '''
#     First get the artist data in a normal table
# '''
%%sql

SELECT  data ->> 'ACTPNOM' as prenom,
        data ->> 'ACTNOM' as nom,
        data ->> 'ANNEENAISS' as yearBirth,
        data ->> 'id_art' as id_art
FROM artistsSQL
;

In [ ]:
# '''
#     Do the same with the films
# '''
%%sql

SELECT  data ->> 'Annee' as yearFilm,
        data -> 'TITRE' ->> 'title' as title,
        data ->> 'MES' as id_art
FROM filmsSQL
;

In [ ]:
# '''
#     Join them. Remember the basic JOIN syntax
#     SELECT table1.column1, table2.column2...
#     FROM table1
#     INNER JOIN table2
#     ON table1.common_filed = table2.common_field;
# '''
%%sql
SELECT title, yearFilm, prenom, nom FROM
    (
        SELECT  data ->> 'ACTPNOM' as prenom,
                data ->> 'ACTNOM' as nom,
                data ->> 'ANNEENAISS' as yearBirth,
                data ->> 'id_art' as id_art
        FROM artistsSQL
    ) as TMP_ART 
    INNER JOIN
    ( 
        SELECT  data ->> 'Annee' as yearFilm,
                data -> 'TITRE' ->> 'title' as title,
                data ->> 'MES' as id_art
        FROM filmsSQL
    ) as TMP_FILMS
    ON TMP_ART.id_art = TMP_FILMS.id_art
;

### Exercise 2.2: For each artist, count the participations on any film (as actor, not as a director)

In [ ]:
%%sql

SELECT prenom, nom, COUNT(*) as num_roles
FROM
    (
        SELECT  title, year,
                roles ->> 'PRENOM' as prenom,
                roles ->> 'NOM' as nom,
                roles ->> 'INTITULE' as intitule
        FROM (
            SELECT  data -> 'TITRE' -> 'title' as title,
                    CAST(data ->> 'Annee' as INTEGER) as year,
                    json_array_elements ( data -> 'ROLES' )  as roles
            FROM filmsSQL
        ) AS TMP
    ) as TMP2
GROUP BY prenom, nom
ORDER BY num_roles DESC
;

###  Exercise 2.3: For each artist, compute the average year of the films in which he/she has participated

In [ ]:
%%sql

SELECT prenom, nom, ROUND(AVG(year),2) as avgYear
FROM
    (
        SELECT  title, year,
                roles ->> 'PRENOM' as prenom,
                roles ->> 'NOM' as nom,
                roles ->> 'INTITULE' as intitule
        FROM (
            SELECT  data -> 'TITRE' ->> 'title' as title,
                    CAST(data ->> 'Annee' as INTEGER) as year,
                    json_array_elements ( data -> 'ROLES' )  as roles
            FROM filmsSQL
        ) AS TMP
    ) as TMP2
GROUP BY prenom, nom
ORDER BY avgYear DESC
;

###  Exercise 2.4: For each artist and each participation in a film, compute the age the artist had when he participated in the movie.

Filter out NaN values !

In [ ]:
%%sql

SELECT title, TMP_FILMS.prenom, TMP_FILMS.nom, TMP_FILMS.yearFilm, yearBirth, yearFilm - yearBirth as age  
FROM
(
    (
        SELECT  data ->> 'ACTPNOM' as prenom,
                data ->> 'ACTNOM' as nom,
                CAST(data ->> 'ANNEENAISS' as INTEGER) as yearBirth,
                data ->> 'id_art' as id_art
        FROM artistsSQL
    ) as TMP_ART 
    INNER JOIN
    ( 
        SELECT  title, yearFilm,
                roles ->> 'PRENOM' as prenom,
                roles ->> 'NOM' as nom,
                roles ->> 'INTITULE' as intitule
        FROM (
                SELECT  data -> 'TITRE' ->> 'title' as title,
                        CAST(data ->> 'Annee' as INTEGER) as yearFilm,
                        json_array_elements ( data -> 'ROLES' )  as roles
                FROM filmsSQL
            ) AS TMP
    ) as TMP_FILMS
    ON TMP_ART.nom = TMP_FILMS.nom AND TMP_ART.prenom = TMP_FILMS.prenom
)
WHERE yearBirth <> double precision 'NaN'
ORDER BY TMP_FILMS.nom, TMP_FILMS.prenom
;

## Exercise 3: New dataset with more complex data

We have worked a lot with the films dataset. Now we will consider a dataset about airline delays in US airports.

In [14]:
!wget "https://raw.githubusercontent.com/lucasgneccoh/BDSS_Dauphine/main/data/airlines.json"
airports_json = json.load(open("airlines.json", "r"))

--2022-02-24 12:04:07--  https://raw.githubusercontent.com/lucasgneccoh/BDSS_Dauphine/main/data/airlines.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4964701 (4.7M) [text/plain]
Saving to: ‘airlines.json.2’

airlines.json.2     100%[===================>]   4.73M  --.-KB/s    in 0.09s   

2022-02-24 12:04:10 (51.0 MB/s) - ‘airlines.json.2’ saved [4964701/4964701]



In [15]:
%%sql

DROP TABLE IF EXISTS airports; 

CREATE TABLE airports (
	id serial NOT NULL PRIMARY KEY,
	airport json NOT NULL,
    time json NOT NULL,
    statistics json NOT NULL
);



 * postgresql+psycopg2://@/postgres


""


In [16]:
# Before populating the table we need to make some adjustments to the names
airports_json_corrected = []
for i, a in enumerate(airports_json):
    aux = {"id": i}
    for k, v in a.items():
        aux[k.lower()] = v
    airports_json_corrected.append(aux)

print(*airports_json_corrected[0])

id airport time statistics


In [17]:
%%capture
import re
for a in airports_json_corrected:
    json_string = re.sub("\'","''", re.sub("Minutes Delayed", "Minutes_Delayed" , json.dumps(a)))
    cmd = f"""INSERT INTO airports SELECT * FROM json_populate_record(null::airports, '{json_string}')""";
    %sql $cmd

Lets see if the table is populated correctly

In [18]:
%%sql

SELECT * FROM airports LIMIT 5;

 * postgresql+psycopg2://@/postgres


,id,airport,time,statistics
0,0,"{'Code': 'ATL', 'Name': 'Atlanta, GA: Hartsfie...","{'Label': '2003/06', 'Month': 6, 'Month Name':...","{'# of Delays': {'Carrier': 1009, 'Late Aircra..."
1,1,"{'Code': 'BOS', 'Name': 'Boston, MA: Logan Int...","{'Label': '2003/06', 'Month': 6, 'Month Name':...","{'# of Delays': {'Carrier': 374, 'Late Aircraf..."
2,2,"{'Code': 'BWI', 'Name': 'Baltimore, MD: Baltim...","{'Label': '2003/06', 'Month': 6, 'Month Name':...","{'# of Delays': {'Carrier': 296, 'Late Aircraf..."
3,3,"{'Code': 'CLT', 'Name': 'Charlotte, NC: Charlo...","{'Label': '2003/06', 'Month': 6, 'Month Name':...","{'# of Delays': {'Carrier': 300, 'Late Aircraf..."
4,4,"{'Code': 'DCA', 'Name': 'Washington, DC: Ronal...","{'Label': '2003/06', 'Month': 6, 'Month Name':...","{'# of Delays': {'Carrier': 283, 'Late Aircraf..."


### Exercise 3.1: Simple data extraction

From this dataset that we have loaded, get a simpler table with the following colums

airport code, airport name, year, month, cancelled flights

***TIP***: Put a LIMIT to the query so that you don't get all the results on the screen

In [19]:
%%sql

SELECT  (airport ->> 'Code' ) as a_code,
        (airport ->> 'Name' ) as a_name,
        (time ->> 'Year' ) as year,
        (time ->> 'Month' ) as month,
        (statistics -> 'Flights' ->> 'Cancelled') as numCancelled
FROM airports
LIMIT 10;

 * postgresql+psycopg2://@/postgres


,a_code,a_name,year,month,numcancelled
0,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",2003,6,216
1,BOS,"Boston, MA: Logan International",2003,6,138
2,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",2003,6,29
3,CLT,"Charlotte, NC: Charlotte Douglas International",2003,6,73
4,DCA,"Washington, DC: Ronald Reagan Washington National",2003,6,74
5,DEN,"Denver, CO: Denver International",2003,6,34
6,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",2003,6,394
7,DTW,"Detroit, MI: Detroit Metro Wayne County",2003,6,123
8,EWR,"Newark, NJ: Newark Liberty International",2003,6,102
9,FLL,"Fort Lauderdale, FL: Fort Lauderdale-Hollywood...",2003,6,13


### Exercise 3.2: Manipulating strings

Notice from the last query that the airport name is formatted in a special way. We want to see the cancelled flights by state, so we need to extract the state from the airport name.

For example from `Boston, MA: Logan International` we want to extract `MA`

Modify the last query to get a column with the state

See this documentation for some tips

https://www.postgresql.org/docs/9.1/functions-string.html


In [26]:
%%sql

SELECT  (airport ->> 'Code' ) as a_code,
        (airport ->> 'Name' ) as a_name,
        substring((airport ->> 'Name' ) from '.*,\s+(.*):.*') as state,
        (time ->> 'Year' ) as year,
        (time ->> 'Month' ) as month,
        (statistics -> 'Flights' ->> 'Cancelled') as numCancelled
FROM airports
LIMIT 10;

 * postgresql+psycopg2://@/postgres


,a_code,a_name,state,year,month,numcancelled
0,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",GA,2003,6,216
1,BOS,"Boston, MA: Logan International",MA,2003,6,138
2,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",MD,2003,6,29
3,CLT,"Charlotte, NC: Charlotte Douglas International",NC,2003,6,73
4,DCA,"Washington, DC: Ronald Reagan Washington National",DC,2003,6,74
5,DEN,"Denver, CO: Denver International",CO,2003,6,34
6,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",TX,2003,6,394
7,DTW,"Detroit, MI: Detroit Metro Wayne County",MI,2003,6,123
8,EWR,"Newark, NJ: Newark Liberty International",NJ,2003,6,102
9,FLL,"Fort Lauderdale, FL: Fort Lauderdale-Hollywood...",FL,2003,6,13


Now with the state in the table, count the number of cancelled flights by state for all periods.

In [30]:
%%sql


SELECT  substring((airport ->> 'Name' ) from '.*,\s+(.*):.*') as state,
        SUM( CAST(statistics -> 'Flights' ->> 'Cancelled' as INTEGER) ) as numCancelled
FROM airports
GROUP BY state
ORDER BY numCancelled DESC
LIMIT 10;

 * postgresql+psycopg2://@/postgres


,state,numcancelled
0,IL,159232
1,TX,111698
2,NY,87900
3,GA,86176
4,CA,80667
5,DC,56170
6,NJ,54880
7,FL,41178
8,MA,38652
9,CO,35926


We got all the cancelled flights in total. But some states have more airports and population than others. A better measure would be to compute the fraction of cancelled flights over all the flights. Do this by state

In [44]:
%%sql

SELECT state, ROUND((numCancelled * 100.0) / total , 2) as fraction
FROM (
    SELECT  substring((airport ->> 'Name' ) from '.*,\s+(.*):.*') as state,
            SUM( CAST(statistics -> 'Flights' ->> 'Cancelled' as DECIMAL) ) as numCancelled,
            SUM( CAST(statistics -> 'Flights' ->> 'Total' as INTEGER) ) as total
    FROM airports
    GROUP BY state
) as TMP
ORDER BY fraction DESC
LIMIT 10;

 * postgresql+psycopg2://@/postgres


,state,fraction
0,NJ,3.29
1,NY,3.19
2,IL,2.98
3,DC,2.68
4,MA,2.59
5,PA,2.02
6,TX,1.86
7,MI,1.73
8,GA,1.69
9,NC,1.54
